In [10]:
import sys
import pickle
import requests
from datetime import datetime
from bs4 import BeautifulSoup

In [11]:
#以下開始設定爬蟲需的函數
def get_date(news_block_node): #從HTML格式中找到news_date的class,取出新聞時間
    date_string = news_block_node.find(class_="news_date").string.split('|')[0][2:-1]
    return(datetime.strptime(date_string, '%Y.%m.%d').strftime('%Y-%m-%d'))
    
def get_title(news_block_node): #從HTML格式中找到news_title的class,取出新聞
    return news_block_node.find(class_="news_title").a.string

def get_link(news_block_node): #從HTML格式中找到news_title的class,再後面找到a href取得網址
    return news_block_node.find(class_="news_title").a.get('href')

def get_content(link): #讀取連結文章內容，且為繁體中文文章故使用UTF-8編碼
    r = requests.get(link) #用request的get方法把該文章網址抓下來
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'html.parser') #指定html為解析器
    article_node = soup.find(itemprop='articleBody')
    article = article_node.get_text() #將文章儲存在article
    return article.replace("\n", "") #將換行"\n刪除"

In [12]:
def get_news_info(each_news):  #用上面設定的函數整合一起抓取資料
    date  = get_date(each_news)
    title = get_title(each_news)
    link  = get_link(each_news)
    content = get_content(link)
    
    info = {'date' : date,
            'title': title,
            'link' : link,
            'content': content}
    return(info)

def get_page_news(page_url):   #設定抓取每個分頁的news blocks的內容
    r = requests.get(page_url) #用request的get方法把該分頁網址抓下來
    r.encoding = "UTF-8" #為繁體中文文章故使用UTF-8編碼

    soup = BeautifulSoup(r.text, 'html.parser')  #指定html為解析器
    news_blocks = soup.find_all(class_="news-list-item clearfix ") #找到html中 news-list-item clearfix的class也就是目標news blocks
    
    news = []
    for each_news in news_blocks: 
        try:
            news_info = get_news_info(each_news)
#             print(get_title(each_news))
        except:
#             print('-------{}-------'.format())
            pass

        news.append(news_info) #設定空白list，並用前述取得新聞標題的方式將區塊中各新聞標題依序放入，且利用except作錯誤處理
    return(news)


def get_new_talk_news(from_page=1, end_page=270, url="https://newtalk.tw/news/subcategory/2/政治/"): #抓取指定主題中指定範圍分頁的新聞
    print("page_number from {} to {}".format(from_page, end_page -1))#格式化字串
    data = []
    for page_number in range(from_page, end_page):
        print("page_number: {}".format(page_number))
        data = data + get_page_news( url+str(page_number) )#將抓取的格式化字串依照頁數遞迴放入所設的空白list中
    
    print('done')
    return(data)

In [6]:
data = get_new_talk_news(from_page=1, end_page=270) #抓取1 ~ 270頁的新聞資料

page_number from 1 to 269
page_number: 1
page_number: 2
page_number: 3
page_number: 4
page_number: 5
page_number: 6
page_number: 7
page_number: 8
page_number: 9
page_number: 10
page_number: 11
page_number: 12
page_number: 13
page_number: 14
page_number: 15
page_number: 16
page_number: 17
page_number: 18
page_number: 19
page_number: 20
page_number: 21
page_number: 22
page_number: 23
page_number: 24
page_number: 25
page_number: 26
page_number: 27
page_number: 28
page_number: 29
page_number: 30
page_number: 31
page_number: 32
page_number: 33
page_number: 34
page_number: 35
page_number: 36
page_number: 37
page_number: 38
page_number: 39
page_number: 40
page_number: 41
page_number: 42
page_number: 43
page_number: 44
page_number: 45
page_number: 46
page_number: 47
page_number: 48
page_number: 49
page_number: 50
page_number: 51
page_number: 52
page_number: 53
page_number: 54
page_number: 55
page_number: 56
page_number: 57
page_number: 58
page_number: 59
page_number: 60
page_number: 61
page_nu

In [13]:
sys.setrecursionlimit(100000) #設定遞迴限制次數
with open('C:/Users/Chien/new_talk.pkl', 'wb') as handle:#在working dictionary 設立檔案空間儲存前面抓取的資料
    pickle.dump(data, handle)

In [17]:
data2 = get_new_talk_news(from_page=1, end_page=3)

page_number from 1 to 2
page_number: 1
page_number: 2
done


In [18]:
print(data2)

[{'date': '2019-03-05', 'title': '肯定張亞中部份看法  王金平：沒有什麼合作不合作', 'link': 'https://newtalk.tw/news/view/2019-03-05/215574', 'content': "宣布參選總統的孫文學校總校長張亞中，今（5）天在協調下，拜會即將宣布角逐2020大選的前立法院長王金平，近35分鐘的會面中，張主動提出立委席次恢復為200席、委託第三方重新檢視人頭黨員、提名人選確定前辦理辯論等意見。至於兩人是否洽談合作關係，王受訪時表示，「沒有什麼合作不合作，要合作就合作，讓國民黨真正變好，未來有機會重新執政。」張亞中在會中提出四點建議，他說，2005年藍綠立委修憲把立委席次減半，導致選舉制度偏向大黨，壓縮小黨，為了台灣的的長治久安，希望能恢復到200席；其次，不管用何種初選制度，希望黨內能舉辦辯論，讓社會了解國民黨在想什麼。張亞中還提到，國民黨內的人頭黨員為人詬病，他鼓勵王金平促成黨委託第三方，比如律師公會，重新檢視人頭黨員議題。另外，黨內諸多「太陽」重提和平協議，也希望各個候選人能說明內容與程序。王金平聞言後，對於立委席次回復到200席，當場表示認同，但對於採取德國制或日本制，王未多做回應。至於人頭黨員問題，王金平則認為現在並不是談論這個問題的時間，也尚未公開兩岸論述的主張。最後，張亞中預告，自己將來會繼續拜會朱立倫、吳敦義，就核心問題進行交談與了解，希望能做到真民調、真黨員投票、真辯論，才能產生真正的太陽。王金平會後受訪時表示，兩人交換一些意見而已。至於有無進一步合作空間，王說，「沒有什麼合作不合作，要合作就合作，讓國民黨真正好，改革好，未來有機會重新執政，好好就憲法等問題，大家再做一番改革。」他說兩岸當然要和平，一定要穩定，對於張亞中非常特殊的看法，他認為非常值得大家了解、吸收，進一步做施政參考。 \r          (function(d,a){d[a]=d[a]||function(){(d[a].q=d[a].q||[]).push(arguments)};}(window,'dable'));\r          dable('renderWidget', 'dablewidget_6Xg8WqoN');\r          \r        雙方閉門會議近35分鐘